<a href="https://colab.research.google.com/github/upinderpalsingh/5pisa/blob/main/niftynext_50___scan__breakout_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

# Function to scrape Nifty Next 50 stocks from Moneycontrol
def get_nifty_next_50_stocks():
    url = "https://www.moneycontrol.com/stocks/marketstats/indexcomp.php?optex=NSE&opttopic=indexcomp&index=38"  # Nifty Next 50 page
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        stocks = []
        for row in soup.select("tr[bgcolor] td:nth-child(1) a"):
            stock_name = row.text.strip()
            stocks.append(stock_name + ".NS")  # Append ".NS" for NSE stocks

        return stocks[:50]  # Limit to 50 stocks for efficiency

    except Exception as e:
        print(f"Error fetching Nifty Next 50 stocks: {e}")
        return []

# Function to check breakout conditions
def check_breakout(stock):
    try:
        df = yf.download(stock, period="6mo", interval="1d")

        if df.empty:
            return None

        df['50_SMA'] = df['Close'].rolling(window=50).mean()
        df['200_SMA'] = df['Close'].rolling(window=200).mean()

        df['Upper_BB'] = df['Close'].rolling(window=20).mean() + (df['Close'].rolling(window=20).std() * 2)
        df['Lower_BB'] = df['Close'].rolling(window=20).mean() - (df['Close'].rolling(window=20).std() * 2)

        df['RSI'] = 100 - (100 / (1 + (df['Close'].diff().clip(lower=0).mean() / df['Close'].diff().clip(upper=0).abs().mean())))

        df['Volume_Surge'] = df['Volume'] / df['Volume'].rolling(window=10).mean()

        # Breakout Criteria
        latest = df.iloc[-1]

        breakout = (latest['Close'] > latest['50_SMA']) and (latest['Close'] > latest['Upper_BB']) and \
                   (latest['RSI'] > 60) and (latest['Volume_Surge'] > 2)

        if breakout:
            return {
                "Stock": stock,
                "Price": latest['Close'],
                "RSI": latest['RSI'],
                "Volume Surge": latest['Volume_Surge'],
                "50 SMA": latest['50_SMA']
            }

    except Exception as e:
        print(f"Error fetching {stock}: {e}")
        return None

# Continuous scanning function (Runs every minute)
def start_breakout_scanner():
    while True:
        print("\n🔍 Scanning for Breakout Stocks in Nifty Next 50...")
        nifty_next_50_stocks = get_nifty_next_50_stocks()  # Fetch updated Nifty Next 50 stocks list

        breakout_stocks = []
        for stock in nifty_next_50_stocks:
            result = check_breakout(stock)
            if result:
                breakout_stocks.append(result)

        breakout_df = pd.DataFrame(breakout_stocks)

        # Display results
        if breakout_df.empty:
            print("🚨 No breakout stocks found.")
        else:
            print("🔥 Breakout Stocks Found 🔥")
            print(breakout_df)

        # Wait for 1 minute before scanning again
        print("\n⏳ Waiting for 1 minute before next scan...\n")
        time.sleep(60)

# Start scanning
start_breakout_scanner()


🔍 Scanning for Breakout Stocks in Nifty Next 50...
🚨 No breakout stocks found.

⏳ Waiting for 1 minute before next scan...


🔍 Scanning for Breakout Stocks in Nifty Next 50...
🚨 No breakout stocks found.

⏳ Waiting for 1 minute before next scan...


🔍 Scanning for Breakout Stocks in Nifty Next 50...
🚨 No breakout stocks found.

⏳ Waiting for 1 minute before next scan...

